Urban Data Science & Smart Cities <br>
URSP688Y Spring 2025<br>
Instructor: Chester Harvey <br>
Urban Studies & Planning <br>
National Center for Smart Growth <br>
University of Maryland

# Exercise02

## Problem

The District Court of Maryland provides data on eviction-related court filings on the [Maryland Open Data Portal](https://opendata.maryland.gov/Housing/District-Court-of-Maryland-Eviction-Case-Data/mvqb-b4hf/about_data), and the Department of Housing and Community Development (DHCD) summarizes these data on a [dashboard](https://app.powerbigov.us/view?r=eyJrIjoiYWI1Yzg0YjYtNDFkZS00MDUyLThlMDctYmE1ZjY5MGI0MWJhIiwidCI6IjdkM2I4ZDAwLWY5YmUtNDZlNy05NDYwLTRlZjJkOGY3MzE0OSJ9&pageName=ReportSection). This dashboard includes some helpful information about filing types and how the eviction process works.

Despite efforts to make eviction data public, the eviction process is complicated and the data are messy and technical, making them difficult to understand. Take, for example, how the table represents evictions: each row represents a court filing, but only some of these filings indicate an eviction (`'Event Type' == 'Warrant of Restitution - Return of Service - Evicted'`).

Please help DHCD summarize the filings to understand how tenants are impacted by the eviction process, including:

1. ***(I demo this below)*** How many unique cases get filed each month?
2. ***(You do)*** What percent of cases end in eviction?
3. ***(You do)*** Do cases of different types (e.g., tennant holding over, breach of lease, failure to pay rent) culminate in eviction at different rates?
4. ***(Optional)*** How long, on average, does it take for cases to move from a petition being filed to warrent of restitution?
5. ***(Optional)*** What is the eviction rate per person per year at the county level? Which county has a higher rate?
6. ***(Optional)*** Are there other interesting ways you can summarize these data?

To address these questions, you will need to familiarize yourself with how the data are structured, including what each row represents (see above), what data are stored in each column, how consistently these data are structured into categories, etc. You may have to convert data types, correct inconsistencies in categories or simplify how categories are represented, change names of columns so they're intuitive, or do other data wrangling to make the data easier to work with. Then you will have to group/aggregate the data by one or more dimensions to make summary calculations.

I recommend approaching this in two phases: 

- First, write a general-purpose function that takes the raw table as an input——you can decide whether the argument to this function is a path to the CSV or a Pandas DataFrame into which you have already the CSV--and cleans it up into a tidy dataframe that is easy to use for the second phase and for future analyses. You can imagine using this function as a handy pre-processor whenever you download an updated copy of the eviction data. ***I started writing this function for you below, but you can modify or add to it to fit your purposes.***

- Second, write multiple single-purpose functions (though there may be opportunities to generalize) that take your tidy dataframe as an input and do calculations with it to address each of the questions above. ***I wrote an example function to answer the first question above—how many unique cases get filed each month? You try writing functions for questions 2–3. 4–6 are optional if you want an extra challenge.***

- You may also write additional helper functions that are called by your main functions.

## Data

Included in this folder are the three CSVs I used in Demo 4: 
- `District_Court_of_Maryland_Eviction_Case_Data_MG_PG.csv`: Court filings for Montgomery and Prince George's County. I recommend starting with these filings as a 'minimal example.' Ideally, your code will be able to handle filings from other counties or the whole state if they are provided in the same format. You are welcome to try using data from the whole state, but it's not required. ***To economize storage space on GitHub, please don't commit statewide data in your pull request.***
- `acs2023_5yr_B01003_mg.csv`: Zipcode-level populations from the American Community Survey (2023 5-year estimates) for Montgomery County, downloaded from [Census Reporter](https://censusreporter.org/data/table/?table=B01003&geo_ids=05000US24031,860|05000US24031&primary_geo_id=05000US24031)
- `acs2023_5yr_B01003_pg.csv`: Zipcode-level populations from the American Community Survey (2023 5-year estimates) for Prince George's County, downloaded from [Census Reporter](https://censusreporter.org/data/table/?table=B01003&geo_ids=05000US24033,860|05000US24033&primary_geo_id=05000US24033)

## File Management and Submitting
1. In GitHub Desktop, **make a branch** in your fork of the course respository. Give the branch a name that will help you keep track of what you will commit to it (e.g., "exercise2"). [Here's some documentation about managing branches in GitHub Desktop](https://docs.github.com/en/desktop/making-changes-in-a-branch/managing-branches-in-github-desktop)
2. Switch to your new branch. If you already started working on Exercise 2, you can choose the option to bring changes to the new branch.
3. Save a copy of the exercise notebook with your first name as an underscored suffix (e.g., `exercise02_chester.ipynb`)
4. Commit and push your saved notebook to your branch.
5. Make sure that the only files included in your pull request are those you added for this exercise.
6. Make a pull request from your branch.

## Getting Started
Let's get started working on this together, based on the code we wrote in Demo 4.

### Phase -1: Dependencies

In [2]:
# Import dependencies--external packages our code will depend on
import pandas as pd

In [3]:
# Define function to make a string all lowercase and replace spaces with underscores
# (Can you see where I use this helper function several times below?)
def lower_underscore(string):
    return string.lower().replace(' ','_')

### Phase 0: Load Data

In [4]:
# Load the filings from CSV
df_filings_raw = pd.read_csv('District_Court_of_Maryland_Eviction_Case_Data_MG_PG.csv')

In [5]:
# Explore the raw dataframe. How would it be useful to tidy it up?
df_filings_raw.head(10)

,Unnamed: 0,Event Date,Event Type,Event Comment,County,Location,Tenant City,Tenant State,Tenant ZIP Code,Case Type,Case Number,Evicted Date,Event Year,Eviction Year
0,0,01/03/2023,Warrant of Restitution - Return of Service - E...,NaN,Montgomery,Rockville,Silver Spring,MD,20910.0,Failure to Pay Rent,D-061-LT-22-004107,12/08/2022,2023.0,2022.0
1,1,01/03/2023,Warrant of Restitution - Return of Service - E...,NaN,Montgomery,Rockville,Silver Spring,MD,20910.0,Failure to Pay Rent,D-061-LT-22-000755,12/08/2022,2023.0,2022.0
2,2,01/03/2023,Warrant of Restitution - Return of Service - E...,NaN,Montgomery,Rockville,Bethesda,MD,20814.0,Failure to Pay Rent,D-061-LT-22-000816,12/07/2022,2023.0,2022.0
3,3,01/03/2023,Warrant of Restitution - Return of Service - E...,NaN,Montgomery,Rockville,Silver Spring,MD,20902.0,Failure to Pay Rent,D-061-LT-22-006362,12/08/2022,2023.0,2022.0
4,4,01/03/2023,Warrant of Restitution - Return of Service - E...,NaN,Montgomery,Rockville,Rockville,MD,20850.0,Failure to Pay Rent,D-061-LT-22-004268,12/07/2022,2023.0,2022.0
5,5,01/03/2023,Warrant of Restitution - Return of Service - E...,NaN,Montgomery,Rockville,Rockville,MD,20850.0,Failure to Pay Rent,D-061-LT-22-004289,12/07/2022,2023.0,2022.0
6,6,01/03/2023,Warrant of Restitution - Return of Service - E...,NaN,Montgomery,Rockville,Silver Spring,MD,20902.0,Failure to Pay Rent,D-061-LT-22-006357,12/08/2022,2023.0,2022.0
7,7,01/03/2023,Warrant of Restitution - Return of Service - E...,NaN,Montgomery,Rockville,Silver Spring,MD,20906.0,Failure to Pay Rent,D-061-LT-22-015154,01/03/2023,2023.0,2023.0
8,8,01/03/2023,Warrant of Restitution - Return of Service - E...,NaN,Montgomery,Rockville,Silver Spring,MD,20906.0,Failure to Pay Rent,D-061-LT-22-015132,01/03/2023,2023.0,2023.0
9,9,01/03/2023,Warrant of Restitution - Return of Service - E...,NaN,Montgomery,Rockville,Silver Spring,MD,20906.0,Failure to Pay Rent,D-061-LT-22-002958,12/02/2022,2023.0,2022.0


### Phase 1: Make a Tidy DataFrame

In [6]:
# Function clean up filings into a tidy dataframe
def tidy_court_filings(df):
    """Make a tidy DataFrame of court filings
    
    Input a Pandas DataFrame from a CSV downloaded at
    https://opendata.maryland.gov/Housing/District-Court-of-Maryland-Eviction-Case-Data/mvqb-b4hf/about_data
    """
    # Drop unnecessary columns
    df = df.drop(columns=['Unnamed: 0'])
    
    # Convert column names to all lowercase with underscores instead of spaces
    # (This is a person style, but I find it easier to operate with column names that are simple and don't have spaces.)
    # One way to do this is with a for loop:
    new_column_names = []
    for col in df.columns:
        col = lower_underscore(col)
        new_column_names.append(col)
    df.columns = new_column_names
      
    # Make sure dates are stored as datetimes instead of strings
    df['event_date'] = pd.to_datetime(df['event_date'])
    df['evicted_date'] = pd.to_datetime(df['evicted_date'])

    # Make sure zip codes and years are integers
    # (Can you think of a situation where we wouldn't want to store them as integers?)
    df['tenant_zip_code'] = df['tenant_zip_code'].astype('Int64')
    df['tenant_zip_code'] = df['tenant_zip_code'].astype('Int64')

    # Make sure event types are consistently classified and use simple class names
    # (You can use the `map` method with a dictionary as the argument to reclassify
    # every value in a column that equals a key in the dictionary with value for that key.)
    reclassifier = {
        'Petition - For Warrant of Restitution Filed': 'petition',
        'petition - For Warrant of Restitution Filed': 'petition',
        'Warrant of Restitution - Return of Service - Cancelled': 'warrant_cancelled',
        'Warrant of Restitution - Return of Service - Evicted': 'warrant_evicted',
        'Warrant of Restitution - Return of Service - Expired': 'warrant_expired',
    }
    df['event_type'] = df['event_type'].map(reclassifier)

    # Make sure case types are consistently classified and use simple class names
    reclassifier = {}
    for case_type in df['case_type']:
        reclassifier[case_type] = lower_underscore(case_type)
    df['case_type'] = df['case_type'].map(reclassifier)

    # Make sure city names are consistently title case
    df['tenant_city'] = df['tenant_city'].str.title()

    # Recalculate event year and eviction year based on dates, just to be confident they're accurate
    # and make sure they're stored as integers
    df['event_year'] = df['event_date'].dt.year.astype('Int64')
    df['eviction_year'] = df['evicted_date'].dt.year.astype('Int64')

    # Change 'evicted_date' to 'eviction_date' for grammatical consistency with 'eviction_year'
    # (Okay, now I'm getting picky, but you can see how you can get into the details with making
    # a dataframe nice and tidy)
    df = df.rename(columns={'evicted_date': 'eviction_date'})

    # Put columns in a more intuitive order
    # (Again, highly optional, but can be useful for tidying)
    df = df[[
        'case_number',
        'case_type',
        'event_date',
        'event_year',
        'event_type',
        'event_comment',
        'eviction_date',
        'eviction_year',
        'county',
        # 'location', # I believe 'location' refers to the district court location, which isn't important to us, so let's leave it out
        'tenant_city',
        'tenant_state',
        'tenant_zip_code',
    ]]    

    return df

df_filings_tidy = tidy_court_filings(df_filings_raw)

df_filings_tidy.head(2)

,case_number,case_type,event_date,event_year,event_type,event_comment,eviction_date,eviction_year,county,tenant_city,tenant_state,tenant_zip_code
0,D-061-LT-22-004107,failure_to_pay_rent,2023-01-03,2023,warrant_evicted,NaN,2022-12-08,2022,Montgomery,Silver Spring,MD,20910
1,D-061-LT-22-000755,failure_to_pay_rent,2023-01-03,2023,warrant_evicted,NaN,2022-12-08,2022,Montgomery,Silver Spring,MD,20910


### Phase 2: Analyze Tidy DataFrame to Answer Questions

#### 1. How many unique cases get filed each month?

In [7]:
def unique_cases_per_month(df, distinguish_year=True):
    """Calculate unique cases filed in each calendar month

    df: Pandas DataFrame pre-processed by the `tidy_court_filings` function

    distinguish_year: True or False (default: True)
        If True, counts are broken out by year and month
        If False, counts from all years are combined in like calendar months
    """
    # Make sure that earliest events are first
    df = df.sort_values('event_date')
    # Keep only the first row for each case number
    unique_cases = df.drop_duplicates('case_number', keep='first')

    # Count cases in each month and year
    # adapted from https://stackoverflow.com/questions/38792122/how-to-group-and-count-rows-by-month-and-year-using-pandas
    if distinguish_year:
        case_counts = unique_cases.groupby([
            unique_cases['event_date'].dt.year.rename('year'), 
            unique_cases['event_date'].dt.month.rename('month')
        ])['case_number'].count()
    else:
        case_counts = unique_cases.groupby([ 
            unique_cases['event_date'].dt.month.rename('month')
        ])['case_number'].count()

    # Structure output in a dataframe with months and years as columns
    case_counts = pd.DataFrame(case_counts).reset_index()
    
    return case_counts
 

In [8]:
unique_cases_per_month(df_filings_tidy)

,year,month,case_number
0,2023,1,483
1,2023,2,242
2,2023,3,395
3,2023,4,251
4,2023,5,339
5,2023,6,1281
6,2023,7,7877
7,2023,8,5109
8,2023,9,2282
9,2023,10,1847


In [9]:
unique_cases_per_month(df_filings_tidy, distinguish_year=False)

,month,case_number
0,1,2329
1,2,4105
2,3,4870
3,4,3791
4,5,4206
5,6,4445
6,7,12089
7,8,9728
8,9,6833
9,10,5143


#### 2. What percent of cases end in eviction?

In [10]:
#Find the listings/options for types of event type (‘event type’ variable) and total number per different types of events
df_filings_tidy['event_type'].value_counts()


event_type
petition             61818
warrant_cancelled    22848
warrant_evicted       8138
warrant_expired       3273
Name: count, dtype: int64

In [11]:
#Find the sum of all event types by formatting a new dataframe and defining the total sum

data = {
    'event_type': ['petition', 'warrant_cancelled', 'warrant_evicted', 'warrant_expired'],
    'count': [61818, 22848, 8138, 3273]
}

df = pd.DataFrame(data)

total_sumQ2 = df['count'].sum()

print("Total sum of event counts:", total_sumQ2)

##note: It would be ideal to avoid 'cementing in' event type and count values, will look into how to write alternate code 





Total sum of event counts: 96077


In [12]:
warrant_evicted= 8138 
#note: I needed to define warrant_evicted and will find another way to define warrant_evicted not by an integer, since this is a static definition, and the dataset could change

In [13]:
#Divide total event type by ‘warrent-evicted’ to find proportion 


percent= (warrant_evicted/total_sumQ2)*100
round(percent, 3)  

#8138/96077= 0.08470






np.float64(8.47)

In [14]:
#official print of question 2 finding

print("The percent of cases that end in eviction is", round(percent,3), "%")

The percent of cases that end in eviction is 8.47 %


#### 3. Do cases of different types (e.g., tennant holding over, breach of lease, failure to pay rent) culminate in eviction at different rates?

In [15]:
#Search which variable these different types are listed under (variable: case type) and the different options for case type
#Use similar code to question 2 to find types 


df_filings_tidy['case_type'].value_counts()



case_type
failure_to_pay_rent    90390
tenant_holding_over     3784
wrongful_detainer       1167
breach_of_lease          736
Name: count, dtype: int64

In [16]:
#Find total sum of case types and print result 
data3 = {
    'case_type': ['failure_to_pay_rent', 'tenant_holding_over', 'wrongful_detainer', 'breach_of_lease'],
    'count': [90390, 3784, 1167, 736]
}

df = pd.DataFrame(data3)


total_sumQ3 = df['count'].sum()

print("Total sum of case types:", total_sumQ3)



Total sum of case types: 96077


In [17]:
#Make a new dataframe of all types under case types variable, finding the sum and percentage 

data3 = {
    'case_type': ['failure_to_pay_rent', 'tenant_holding_over', 'wrongful_detainer', 'breach_of_lease'],
    'count': [90390, 3784, 1167, 736],
    'frequency': [90390/total_sumQ3, 3784/total_sumQ3, 1167/total_sumQ3, 736/total_sumQ3]
}


In [18]:
#Format dataframe into a frequency table to highlight and compare frequency of occurrence 

df3 = pd.DataFrame(data3)

total_sumQ3 = df3['count'].sum()

df3['frequency'] = df3['count'] / total_sumQ3


print(df3)

#This tables shows how different case types happen at different frequencies, in general (regardless of yes or no eviction) 



             case_type  count  frequency
0  failure_to_pay_rent  90390   0.940808
1  tenant_holding_over   3784   0.039385
2    wrongful_detainer   1167   0.012147
3      breach_of_lease    736   0.007661


In [57]:

#Now to find case types as they relate to eviction

eviction_results_FTPR = df_filings_tidy.groupby(['case_type', 'event_type']).size()


eviction_results_FTPR = df_filings_tidy[
    (df_filings_tidy['case_type'] == 'failure_to_pay_rent') & 
    (df_filings_tidy['event_type'] == 'warrant_evicted')
].shape[0]

print(eviction_results_FTPR)
#7344 evictions from failure_to_pay_rent; 8.12% of FTPR cases result in eviction
7344/90390

7344


0.08124792565549287

In [58]:
eviction_results_THO = df_filings_tidy.groupby(['case_type', 'event_type']).size()


eviction_results_THO = df_filings_tidy[
    (df_filings_tidy['case_type'] == 'tenant_holding_over') & 
    (df_filings_tidy['event_type'] == 'warrant_evicted')
].shape[0]

print(eviction_results_THO)
#460 evictions from tenant_holding_over; 12% of THO cases result in eviction
460/3784

460


0.12156448202959831

In [59]:
eviction_results_WD = df_filings_tidy.groupby(['case_type', 'event_type']).size()


eviction_results_WD = df_filings_tidy[
    (df_filings_tidy['case_type'] == 'wrongful_detainer') & 
    (df_filings_tidy['event_type'] == 'warrant_evicted')
].shape[0]

print(eviction_results_WD)
#194 evictions from wrongful_detainer; 16% of WD cases result in eviction
194/1167


194


0.1662382176520994

In [60]:
eviction_results_BOL = df_filings_tidy.groupby(['case_type', 'event_type']).size()


eviction_results_BOL = df_filings_tidy[
    (df_filings_tidy['case_type'] == 'breach_of_lease') & 
    (df_filings_tidy['event_type'] == 'warrant_evicted')
].shape[0]

print(eviction_results_BOL)
#140 evictions from breach_of_lease; 19% of BOL cases result in eviction 
140/736

140


0.19021739130434784

In [ ]:
#Overall, failure to pay rent is the most common case type, and most common case type to result in eviction. 
#However, the three other case types have larger percentages of eviction results, due to their lower general frequency but higher chance of resulting in eviction

Question #4 (Optional) How long, on average, does it take for cases to move from a petition being filed to warrent of restitution?

In [19]:
  print(df.columns) #checking the df output

Index(['case_type', 'count'], dtype='object')


In [20]:
#assuming petition being filed= event date
#assuming warrent of restitution= eviction date

#Find a way to calculate the difference between the event data and the evicted date 
#This would require transferring the month/day/year to be countable  (already in pd.to_datetime from previous questions)


df_filings_tidy = tidy_court_filings(df_filings_raw)

df_filings_tidy.head(2)



,case_number,case_type,event_date,event_year,event_type,event_comment,eviction_date,eviction_year,county,tenant_city,tenant_state,tenant_zip_code
0,D-061-LT-22-004107,failure_to_pay_rent,2023-01-03,2023,warrant_evicted,NaN,2022-12-08,2022,Montgomery,Silver Spring,MD,20910
1,D-061-LT-22-000755,failure_to_pay_rent,2023-01-03,2023,warrant_evicted,NaN,2022-12-08,2022,Montgomery,Silver Spring,MD,20910


In [21]:
#Add a new column, days_between, to the df that subtracts the event_date from the eviction_date 
#Could fill NA addresses if date not found 

df_filings_tidy['days_between'] = (df_filings_tidy['eviction_date'] - df_filings_tidy['event_date']) 


#Note: I initially added ".dt.days.fillna(0).astype(int)" to this line, but then found that calculating the mean with NA values would ignore them, rather than count the NA 'zeroes' as values



In [22]:
#Check that the days_between column exists and find values

df_filings_tidy

df_filings_tidy.head(20)['days_between']

0    -26 days
1    -26 days
2    -27 days
3    -26 days
4    -27 days
5    -27 days
6    -26 days
7      0 days
8      0 days
9    -32 days
10   -27 days
11   -27 days
12   -27 days
13   -28 days
14   -28 days
15   -28 days
16   -28 days
17   -28 days
18   -28 days
19   -28 days
Name: days_between, dtype: timedelta64[ns]

In [23]:
#Find average days_between value and print findings


print (df_filings_tidy['days_between'].mean())



-16 days +08:36:41.228803146


Question #5 (Optional) What is the eviction rate per person per year at the county level? Which county has a higher rate?

In [24]:
#load census data and combine PG and Mo Counties


def load_census_reporter_csv(path):
    return pd.read_csv(path, skiprows=[1])

df_census_mg = load_census_reporter_csv('acs2023_5yr_B01003_mg.csv')#Montgomery County
df_census_pg = load_census_reporter_csv('acs2023_5yr_B01003_pg.csv')#Prince George's County

df_census = pd.concat([df_census_mg, df_census_pg], axis=0)

In [25]:
#Format tables

df_census = df_census.rename(columns={'name':'census_zip', 'B01003001':'population'})
df_census = df_census.drop(columns=['B01003001, Error'])

In [38]:
df_census.head() #to double check df_census output 

,geoid,census_zip,population
0,86000US20705,20705,28119
1,86000US20707,20707,36549
2,86000US20777,20777,3046
3,86000US20812,20812,314
4,86000US20814,20814,30822


In [ ]:
#Total population from dataset: 1057586 (MOCO); 955584 (PGCO) 

In [33]:
#Find proportion of each county within entire county variable

df_filings_tidy['county'].value_counts()


#Need to find population per county from acs2023 data: 
    #acs2023_5yr_B01003_mg.csv Montgomery County
    #acs2023_5yr_B01003_pg.csv Prince George's County




county
Prince George's    67003
Montgomery         29074
Name: count, dtype: int64

In [34]:
df_filings_tidy['event_type'].value_counts()

event_type
petition             61818
warrant_cancelled    22848
warrant_evicted       8138
warrant_expired       3273
Name: count, dtype: int64

In [41]:
#Find eviction rates by each county: 
#Need to filter by county and by event_type


#Montgomery 
event_counts_MO = df_filings_tidy.groupby(['event_type', 'county']).size()

event_counts_MO = df_filings_tidy[
    (df_filings_tidy['event_type'] == 'warrant_expired') & 
    (df_filings_tidy['county'] == 'Montgomery')
].shape[0]

print(event_counts_MO)
#1541 evictions


1541


In [61]:
#Calculate eviction per person within year 

MOCO_population= 1057586


(event_counts_MO/MOCO_population)


#0.145%

0.0014570919055282502

In [45]:
#Prince George's County 

event_counts_PG = df_filings_tidy.groupby(['event_type', 'county']).size()

#Need to take out apostrophe from Prince George's title

df_filings_tidy['county'] = df_filings_tidy['county'].str.replace("'", "", regex=True)


event_counts_PG = df_filings_tidy[
    (df_filings_tidy['event_type'] == 'warrant_expired') & 
    (df_filings_tidy['county'] == 'Prince Georges')
].shape[0]

print(event_counts_PG)


1732


In [62]:
#Calculate eviction per person within year 

PGCO_population= 955584


(event_counts_PG/PGCO_population)

#0.18% 

0.0018125041859219074

In [ ]:
#Prince George's County has a higher rate of evictions per person, per year, and a lower population than Montgomery County

Question #6 (Optional) Are there other interesting ways you can summarize these data?

In [ ]:
#This data could also compare changes in eviction by year, as well as if particular case types are more common in some areas than other areas